### テスト用のNoote book

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

### Topatches

In [6]:
class ToPatches(nn.Module):
    """1枚の画像を小さなパッチに分割する

    Attributes:
        patch_size: パッチ1辺のサイズ(2)
        projection: パッチデータを線形変換する全結合層
    """
    def __init__(self, in_channels, dim, patch_size):
        """
        Args:
            in_channels(int): 入力画像のチャネル数(in_channels=3)
            dim(int): 線形変換後のパッチデータの次元(dim=128)
            patch_size(int): パッチ1辺のサイズ(patch_size=2)
        """
        super().__init__()
        self.patch_size = patch_size
        # チャネル数にパッチ1辺のサイズの2乗を掛けてパッチ1個のサイズを計算
        # 3 * 2 * 2 = 12
        patch_dim = in_channels * patch_size * patch_size
        # 入力サイズをパッチのサイズ(12)
        # ユニット数をdim(128)に設定した全結合層を作成
        self.projection = nn.Linear(patch_dim, dim)
        # 正規化を行う
        self.norm = nn.LayerNorm(dim)

    def forward(self, x):
        """パッチに分割する一連の順伝播処理
        Args:
            x(torch.Tensor): 入力画像
        Returns:
            torch.Tensor: パッチに分割されたテンソル
        """
        # xの形状: (バッチサイズ(bs), チャネル数(3), 画像の高さ(32), 画像の幅(32))
        # F.unfoldでパッチに分割
        # kernel_size: パッチ1辺のサイズ(2)
        # stride: パッチ1辺のサイズ(2)
        # F.unfoldの出力の形状: (パッチのサイズ(2*2*3), パッチ数(256),バッチサイズ(bs))
        # movedim(1, -1)でバッチサイズとパッチ数を入れ替え
        # (バッチサイズ(bs), パッチ数(256), パッチのサイズ(2*2*3))となる
        x = F.unfold(x, kernel_size=self.patch_size, stride=self.patch_size).movedim(1, -1)
        # パッチデータを線形変換
        # 出力の形状: (バッチサイズ(bs), パッチ数(256), パッチのサイズ(128))
        x = self.projection(x)
        # 正規化を行う
        x = self.norm(x)
        return x 

In [7]:
topatches = ToPatches(3, 128, 2)
x = torch.randn(8, 3, 32, 32)
out = topatches(x)
# [32, 256, 128]
out.shape

torch.Size([8, 256, 128])

### split_windows

In [3]:
def split_windows(x, window_size):
    """window_sizeに基づいてバッチテンソルxをウィンドウに分割

    Args:
        x: バッチテンソル(bs, パッチ数(行), パッチ数(列), パッチサイズ)
        window_size: ウィンドウ1辺のサイズ(window_size=4)
    Returns:
        ウィンドウに分割されたテンソル
        (bs * ウィンドウの数, ウィンドウ内のパッチ数(window_size**2), パッチサイズ)
    """ 
    # 行方向のウィンドウ数n_hと列方向のウィンドウ数n_wを計算
    n_h, n_w = x.size(1) // window_size, x.size(2) // window_size
    # パッチ行列をウィドウに分割する
    # x.unflatten(1, (n_h, window_size))で[bs, パッチ数(行),\
    # パッチ数(列), パッチサイズ ]が
    # [bs,ウィドウ数(行), ウィドウサイズ(行), パッチ数, パッチサイズ]に変換される
    # .unflatten(-2, (n_w, window_size))で
    # [bs,ウィドウ数(行), ウィドウサイズ(行), パッチ数, パッチサイズ]が
    # [bs, ウィドウ数(行), ウィドウサイズ(行),\
    #  ウィドウ数(列), ウィドウサイズ(列), パッチサイズ]に変換される 
    x = x.unflatten(1, (n_h, window_size)).unflatten(-2, (n_w, window_size))
    # 第3次元(window_size)と第4次元(ウィドウ数(列))を入れ替えて
    # 第一次元、第二次元、第三次元をフラット化
    # x.transpose(2, 3)で[bs, ウィドウ数(行), ウィドウサイズ(行),ウィドウ数(列), ウィドウサイズ(列), パッチサイズ]が
    # [bs, ウィドウ数(行), ウィンドウ数(列), ウィドウサイズ(行), ウィドウサイズ(列), パッチサイズ]に変換される
    # .flatten(0, 2)で[bs, ウィドウ数(行), ウィンドウ数(列), ウィドウサイズ(行), ウィドウサイズ(列), パッチサイズ]が
    # [bs * ウィンドウの数(行 * 列), ウィドウサイズ(行), ウィドウサイズ(列), パッチサイズ]に変換される
    x = x.transpose(2, 3).flatten(0, 2)
    # [bs * ウィンドウの数(行 * 列), ウィドウサイズ(行), ウィドウサイズ(列), パッチサイズ]が
    # [bs * ウィンドウの数(行 * 列), ウィドウ内パッチ数, パッチサイズ]
    x = x.flatten(-3, -2)
    return x

In [4]:
window_size = 4

x = torch.randn(32, 16, 16, 2)
# 返り値 : [32*16=512, 16, 2]
x = split_windows(x, window_size)
x.shape

torch.Size([512, 16, 2])